In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, prepare_model_for_kbit_training
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer , BitsAndBytesConfig
from peft import LoraConfig, get_peft_model , prepare_model_for_kbit_training
import torch

# ตั้งค่า paths
model_path = "EXt1/KMUTT-CPE35-thai-mt5base-summarizer"  # โฟลเดอร์ที่บันทึก adapter
base_model_name = "google/mt5-base"  # ต้องตรงกับโมเดลต้นทางที่ใช้ฝึก

# ตั้งค่า Quantization Config (ต้องตรงกับตอนฝึก)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=False,
    load_in_8bit = True,
    bnb_4bit_use_double_quant=False,
    llm_int8_threshold = 6.0
)

# กำหนด device_map แบบ custom (สามารถปรับได้ตาม VRAM ของ GPU)
device_map = "auto"  # หรือปรับเป็น dictionary ถ้าต้องการระบุชัดเจน
# โหลด Base Model ใหม่ในโหมด 8-bit พร้อม offloading
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    base_model_name,
    quantization_config=quantization_config,
    device_map={"": 0}
)
tokenizer = AutoTokenizer.from_pretrained("EXt1/KMUTT-CPE35-thai-mt5base-summarizer")
# โหลด Adapter ที่ฝึกไว้
model = PeftModel.from_pretrained(
    base_model,
    model_path,
    device_map={"": 0}
)

model = model.merge_and_unload()
model.eval()


C:\Users\Janejojija\AppData\Roaming\Python\Python312\site-packages\peft\tuners\lora\bnb.py:85: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


model.safetensors:  16%|#6        | 377M/2.33G [00:00<?, ?B/s]

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear8bitLt(in_features=768, out_features=768, bias=False)
              (k): Linear8bitLt(in_features=768, out_features=768, bias=False)
              (v): Linear8bitLt(in_features=768, out_features=768, bias=False)
              (o): Linear8bitLt(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear8bitLt(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear8bitLt(in_features=768, out_

C:\Users\Janejojija\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Janejojija\.cache\huggingface\hub\models--google--mt5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [2]:
print(f"ขนาดของโมเดลหลัง Apply Quantization: {model.get_memory_footprint() / 1e9:.2f} GB")
# check_flash_attention(model)

ขนาดของโมเดลหลัง Apply Quantization: 1.08 GB


In [10]:
sample_text = """ประเทศไทย- หรือชื่อทางการว่า ราชอาณาจักรไทย เป็นรัฐชาติอันตั้งอยู่บนคาบสมุทรอินโดจีนและมลายู ในภูมิภาคเอเชียตะวันออกเฉียงใต้ มีพรมแดนด้านตะวันออกติดประเทศลาวและประเทศกัมพูชา ทิศใต้เป็นแดนต่อแดนประเทศมาเลเซียและอ่าวไทย ทิศตะวันตกติดทะเลอันดามันและประเทศพม่า และทิศเหนือติดประเทศพม่าและลาว มีแม่น้ำโขงกั้นเป็นบางช่วง ปกครองด้วยระบอบประชาธิปไตยแบบมีรัฐสภา มีศูนย์กลางการบริหารราชการแผ่นดินอยู่ที่กรุงเทพมหานคร และการปกครองส่วนภูมิภาค จัดระเบียบเป็น 76 จังหวัด
ประเทศไทยมีขนาดใหญ่เป็นอันดับที่ 50 ของโลก มีเนื้อที่ 513,115 ตารางกิโลเมตร และมีประชากรมากเป็นอันดับที่ 20 ของโลก คือ ประมาณ 66 ล้านคน กับทั้งยังเป็นประเทศอุตสาหกรรมใหม่ โดยมีรายได้หลักจากภาคอุตสาหกรรมและการบริการ ไทยมีแหล่งท่องเที่ยวที่มีชื่อเสียงเป็นอันมาก อาทิ พัทยา, ภูเก็ต, กรุงเทพมหานคร และเชียงใหม่ ซึ่งสร้างรายได้ให้แก่ประเทศ เช่นเดียวกับการส่งออกอันมีส่วนสำคัญในการพัฒนาเศรษฐกิจและด้วยจีดีพีของประเทศ ซึ่งมีมูลค่าราว 334,026 ล้านดอลลาร์สหรัฐ ตามที่ประมาณใน พ.ศ. 2553 เศรษฐกิจของประเทศไทยนับว่าใหญ่เป็นอันดับที่ 30 ของโลก
ในอาณาเขตประเทศไทย พบหลักฐานของมนุษย์ซึ่งมีอายุเก่าแก่ที่สุดถึงห้าแสนปี นักประวัติศาสตร์มักถือว่าอาณาจักรสุโขทัยเป็นจุดเริ่มต้นของประวัติศาสตร์ไทย ซึ่งต่อมาตกอยู่ในอิทธิพลของอาณาจักรอยุธยา อันมีความยิ่งใหญ่กว่า และมีการติดต่อกับชาติตะวันตก อาณาจักรอยุธยามีอายุยืนยาว 417 ปีก็เสื่อมอำนาจและล่มสลายไปโดยสิ้นเชิง สมเด็จพระเจ้ากรุงธนบุรีทรงกอบกู้เอกราชและสถาปนาอาณาจักรธนบุรี เหตุการณ์ความวุ่นวายในช่วงปลายอาณาจักร นำไปสู่ยุคสมัยของราชวงศ์จักรีแห่งกรุงรัตนโกสินทร์
ช่วงต้นกรุง ประเทศเผชิญภัยคุกคามจากชาติใกล้เคียง แต่หลังรัชสมัยพระบาทสมเด็จพระจอมเกล้าเจ้าอยู่หัวเป็นต้นมา ชาติตะวันตกเริ่มมีอิทธิพลในภูมิภาคเป็นอย่างมาก นำไปสู่การเข้าเป็นภาคีแห่งสนธิสัญญาหลายฉบับ และการเสียดินแดนบางส่วน กระนั้น ไทยก็ยังธำรงตนมิได้เป็นอาณานิคมของชาติใด ๆ ต่อมาจนช่วงสงครามโลกครั้งที่หนึ่ง ไทยได้เข้าร่วมกับฝ่ายสัมพันธมิตร และในปี พ.ศ. 2475 ได้มีการปฏิวัติเปลี่ยนแปลงจากระบอบสมบูรณาญาสิทธิราชมาเป็นประชาธิปไตย และไทยได้เข้ากับฝ่ายอักษะในระหว่างสงครามโลกครั้งที่สอง จนช่วงสงครามเย็น ไทยได้ดำเนินนโยบายเป็นพันธมิตรกับสหรัฐอเมริกา ทหารเข้ามามีบทบาทในการเมืองไทยอย่างมากหลังปฏิวัติสยามอยู่หลายสิบปี กระทั่งมีการตั้งรัฐบาลพลเรือน และเข้าสู่ยุคโลกเสรีในปัจจุบัน
"""
# Tokenize the input text
inputs = tokenizer(sample_text, return_tensors="pt", max_length=512, truncation=True)

# สร้างการสรุป
with torch.no_grad():
    summary_ids = model.generate(
        inputs["input_ids"].to(model.device),
        max_new_tokens=256,  # กำหนดความยาวของสรุปที่ต้องการ
        num_beams=4,
        early_stopping=True
    )

# Decode the generated summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

ประเทศไทย- หรือชื่อทางการว่า ราชอาณาจักรไทย เป็นรัฐชาติอันตั้งอยู่บนคาบสมุทรอินโดจีนและมลายู ในภูมิภาคเอเชียตะวันออกเฉียงใต้ มีเนื้อที่ 513,115 ตารางกิโลเมตร และมีประชากรมากเป็นอันดับที่ 20 ของโลก
